In [1]:
import math
import timeit

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
def pixel_classifier_scan(sampleName):
    """
    This function takes in the name of a sample and calls the pixel classification method that looks only at the values
    of the pixel's mechanical properties.
    
    Inputs: sampleName - a string of the sample's name, excluding filetype extension. Assumes .txt filetype
    
    Outputs: pixel_identities - a 2D array of the classifications of the each pixel within the scan
    
            *writes a 2D .txt file of the original data + the new classifications
    """
    
    scan = np.loadtxt('../Data/AFM/AggregatedData/%s.txt'% (sampleName))
    
    x2, z = scan.shape
    x = y = int(math.sqrt(x2))
    scan = scan.reshape((x, y, z))
    
    pixel_identities = np.empty((x, y))
    
    for i in range(x):
        for j in range(y):
            pixel_identities[i, j] = pixel_classifier_pixel(scan[i, j])
            
    
    if z == 20:
        scan = scan.reshape((x2, z))
        pixel_identities_1d = pixel_identities.reshape((x2, 1))
        
        scan_and_classif = np.append(scan, pixel_identities_1d, axis = 1)
        np.savetxt('../Data/AFM/DIRECTShowcase/%s.txt'%(sampleName), scan_and_classif) 

    else:
        for i in range(x):
            for j in range(y):
                scan[i, j, 20] = pixel_identities[i, j]
        scan = scan.reshape((x2, z))
        np.savetxt('../Data/AFM/DIRECTShowcase/%s.txt'%(sampleName), scan)
            
    return pixel_identities
    

In [7]:
def pixel_classifier_pixel(pixel):
    """
    This function takes in a 1D array of AFM data from a single scan. It iterates through each xy location and
    classifies that pixel as either crystalline or amorphous, depending on its z values.
    
    inputs: pixel - a 1D array of AFM data for each pixel
    
    outputs: classified_pixel - an integer that contains the pixel's identity, 1 denotes crystalline, -1 denotes
                    amorphous.
    """
    
    adh_cut = 0.033 #greater than 33 mV
    def_cut = 0.0000000026 #less than 2.6 nm
    dis_cut = 0.033 #greater than 33 mV
    mod_cut = -0.03 #greater than -0.03 V
    stif_cut = 0.01 #greater than 0.01 V
    
    adh_sqrd_cut = adh_cut * adh_cut
    def_sqrd_cut = def_cut * def_cut
    dis_sqrd_cut = dis_cut * dis_cut
    mod_sqrd_cut = mod_cut * mod_cut * -1
    stif_sqrd_cut = stif_cut * stif_cut
    
    adh_log_cut = np.log10(adh_cut)
    def_log_cut = np.log10(def_cut)
    dis_log_cut = np.log10(dis_cut)
    mod_log_cut = -1 * np.log10(mod_cut * -1)
    stif_log_cut = np.log10(stif_cut)
    
    adh_inverse_cut = 1 / adh_cut
    def_inverse_cut = 1 / def_cut
    dis_inverse_cut = 1 / dis_cut
    mod_inverse_cut = 1 / mod_cut
    stif_inverse_cut = 1 / stif_cut
    
    cuts = [adh_cut, def_cut, dis_cut, mod_cut, stif_cut, adh_sqrd_cut, def_sqrd_cut, dis_sqrd_cut, mod_sqrd_cut, 
           stif_sqrd_cut, adh_log_cut, def_log_cut, dis_log_cut, mod_log_cut, stif_log_cut, adh_inverse_cut, 
           def_inverse_cut, dis_inverse_cut, mod_inverse_cut, stif_inverse_cut]
    
    classified_pixel = 0
    
    crystalline_count = 0
    
    pixelinfo = pixel * np.array([1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1])
    
    for i in range(20):
        if pixelinfo[i] >= cuts[i]:
            crystalline_cout += 1
        else:
            pass

    if crystalline_count >= 12:
        classified_pixel = 1
    else:
        classified_pixel = -1
    
    return classified_pixel

In [8]:
test = pixel_classifier_scan('20-120')

In [9]:
print (test)

[[-1.  1. -1. ..., -1. -1. -1.]
 [ 1.  1.  1. ..., -1. -1. -1.]
 [ 1.  1. -1. ..., -1. -1. -1.]
 ..., 
 [ 1.  1.  1. ..., -1. -1. -1.]
 [-1. -1. -1. ..., -1. -1. -1.]
 [-1. -1. -1. ..., -1. -1. -1.]]
